In [1]:
import pipeline_objects
from pipeline_objects.data_loader import DataLoader
from pipeline_objects.preprocessor import DataPreprocessor
from pipeline_objects.modeling import RandomForestModel


In [2]:
data_path = pipeline_objects.__path__[0].replace("pipeline_objects", "Data")
loader = DataLoader(root_path = data_path, objects= ['open_results','descriptions','benchmark_stats'])
data = loader.load()

/Users/hassan/Documents/wod-prediction/WOD-prediction/pipeline_objects/data_loader.py:22: DtypeWarning: Columns (2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(self.root_path, file))
/Users/hassan/Documents/wod-prediction/WOD-prediction/pipeline_objects/data_loader.py:22: DtypeWarning: Columns (2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(self.root_path, file))


In [3]:
# to test the pipeline faster, we can sample the data, remove this line to use all the data
data['open_results'] = data['open_results'].sample(1000)

In [4]:
preprocessing_config = {
    'open_results':
    {
        'create_description_embeddings' : False,
    },
    # 'workout_descriptions':{},
    'benchmark_stats':{
        'remove_outliers': True,
        'missing_method':'zero'
    },
    # 'athlete_info':{}
}
preprocessor = DataPreprocessor(config=preprocessing_config)
preprocessed_data = preprocessor.transform(data=data)


/Users/hassan/Documents/wod-prediction/WOD-prediction/pipeline_objects/feature_engineering_parts/helpers.py:338: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_modified[workout_name] = pd.to_numeric(df_modified[workout_name], errors="ignore")


In [ ]:
rf_modeler = RandomForestModel(n_estimators = 10)
rf_modeler.fit(X = preprocessed_data['X'], y = preprocessed_data['y'])

rf_modeler.show_results()